In [63]:
# -*- coding: utf-8 -*-
# 埼玉千葉神奈川記事反映本番用
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import requests
import urllib.request as req
import glob
import json
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
import os
from dotenv import load_dotenv
from selenium.webdriver.common.by import By

print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()

def syuzai_df():
    global df, gs, SPREADSHEET_KEY
    SCOPE = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUT_FILE, SCOPE)
    gs = gspread.authorize(credentials)
    SPREADSHEET_KEY = os.environ['SPREADSHEET_KEY']
    worksheet = gs.open_by_key(SPREADSHEET_KEY).worksheet('master')
    df = pd.DataFrame(worksheet.get_all_values())
    df.columns = list(df.loc[0, :])
    df.drop(0, inplace=True)
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    return df



def recommend_image(write_image_context, length):
    global context_1, save_image_jpg,output_path

    # 元画像を読み込んでくる
    save_image_jpg = f"image/全媒体/eva_board_{tomorrow_url}.jpg"
    output_path = f"eva_board_{tomorrow_url}.jpg"
    # image_path = r"C:\Users\81801\Desktop\twitter_anarytics_bot\recommend_syuzai_report\board_image.jpg" #win
    image_path = r"image/全媒体/eva_board.jpg"  # mac

    image = Image.open(image_path)

    draw = ImageDraw.Draw(image)

    # フォントを指定する（フォントファイルはWindows10ならC:\\Windows\\Fontsにあります）

    # フォントの読み込


    font_path = r"font/LightNovelPOPv2.otf"
        # "font/LightNovelPOPv2.otf"
    # font/LightNovelPOPv2.otf

    # sizeは文字サイズです（とりあえず適当に50）
    font = ImageFont.truetype(font_path, size=40)

    # 文字を描く
    # 最初の(0,0)は文字の描画を開f始する座標位置です　もちろん、(10,10)などでもOK
    # fillはRGBで文字の色を決めています
    draw.multiline_text((55, 40), write_image_context, fill=(255, 255, 255), font=font, spacing=10, stroke_width=5, stroke_fill=(55, 55, 55))

    # 画像を保存する
    image.save(save_image_jpg)
    print('length', length)
    length_croped = 110 + length * 66  # 500
    im = Image.open(save_image_jpg)
    im_crop = im.crop((0, 0, 1000, length_croped))
    im_crop.save(save_image_jpg, quality=100)


def recommend_image2(todoufuken_kanji:str, write_image_context:str, length, n):
    global context_1, save_image_jpg

    # 元画像を読み込んでくる
    save_image_jpg = f"image/全媒体/syuzai_report_{todoufuken_kanji}_{tomorrow_url}_{n}.jpg"
    #save_image_jpg = win_path+f'python/slot/recommend_syuzai_report/{todouhuken}_syuzai_report_{tomorrow_str}.jpg'
    # image_path = r"C:\Users\81801\Desktop\twitter_anarytics_bot\recommend_syuzai_report\board_image.jpg" #win
    image_path ='image/全媒体/board_image.jpg'

    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # フォントを指定する（フォントファイルはWindows10ならC:\\Windows\\Fontsにあります）

    # フォントの読み込
    font_path = "font/LightNovelPOPv2.otf"
    # font/LightNovelPOPv2.otf

    # sizeは文字サイズです（とりあえず適当に50）
    font = ImageFont.truetype(font_path, size=50)

    # 文字を描く
    # 最初の(0,0)は文字の描画を開f始する座標位置です　もちろん、(10,10)などでもOK
    # fillはRGBで文字の色を決めています
    draw.text((135, 585), write_image_context, fill=(255, 255, 255), font=font)

    # 画像を保存する
    image.save(save_image_jpg)
    print('length', length)
    length_croped = 640 + length * 55  # 500
    im = Image.open(save_image_jpg)
    im_crop = im.crop((0, 0, 1300, length_croped))
    im_crop.save(save_image_jpg, quality=100)
    return save_image_jpg


def get_concat_h_blank(im1, im2, color=(27, 44, 74)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def get_concat_v_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def upload_image(in_image_file_name, out_image_file_name):
    if os.path.exists(in_image_file_name):
        with open(in_image_file_name, 'rb') as f:
            binary = f.read()

        data = {
            "name": out_image_file_name,
            "type": 'image/png',
            "overwrite": True,
            "bits": binary
        }

        media_id = wp.call(media.UploadFile(data))['id']
        print(in_image_file_name.split('/')
              [-1], 'Upload Success : id=%s' % media_id)
        return media_id
    else:
        print(in_image_file_name.split('/')[-1], 'NO IMAGE!!')

def post_line(message):
    url = "https://notify-api.line.me/api/notify"
    token = os.environ['LINE_TOKEN']
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    #print('image_path',image_path)
    #files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload) 


def convert_string(x):
    x = x.replace('来店+取材B', '+スロパチ取材').replace('応援地区(ディレクター有)', '').replace('応援地区(ディレクター無)', '').replace('取材C', 'スロパチ潜入取材').replace('+結-MUSUBI-取材', '+スロパチ取材"結"').replace('+取材B', '+スロパチ取材').replace('潜入取材メガテン', 'メガテン').replace('襲来', '').replace('(30％以上)', '').replace('WEB広告', 'スロパチ広告').replace('+トレジャー取材', '+トレジャー').replace('あつまる+スロパチ取材', 'あつまる').replace('潜入光', '光').replace('潜入取材光', '光').replace('取材光', '光').replace('かたまる+スロパチ取材"結"', 'かたまる').replace('光(25％以上30％未満)', '光').replace('スロパチ潜入取材', '潜入取材').replace('あつまるのみ', 'あつまる').replace('応援地区', '').replace('ホールサーチマン金枠', '金枠').replace('ホールサーチマン赤枠', '赤枠').replace('かたまる×スロパチ取材結', 'かたまる').replace('本店館', '本館').replace('ホールサーチマンレインボー枠', 'レインボー枠').replace(' ', '').replace('　', '')
    return x


def tenpo_convert_string(x):
    x = x.replace('相模原ピーくんステージ', 'ピーくんステージ').replace('アミューズメントコミュニティ ', '').replace('店', '').replace('新!', '').replace('本', '本店').replace('新！', '')
    return x


def get_concat_h_multi_blank(im_list):
    _im = im_list.pop(0)
    for im in im_list:
        _im = get_concat_h_blank(_im, im)
    return _im

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

#try:
# WORDPRESS_ID = os.environ['WORDPRESS_ID']
# WORDPRESS_PW = os.environ['WORDPRESS_PW']
# WORDPRESS_URL = os.environ['WORDPRESS_URL']
# wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)

win_path = 'image/全媒体/'
# 埼玉千葉神奈川記事反映本番用
wp_completed_footer = ''
for ken_number in ['23']:
    todouhuken = 'tokyo'
    SERVICE_ACCOUT_FILE =r'sercret_json\twitteranalytics-jsonsercretkey.json'
    read_image_jpg = f"image/全媒体/tokyo_ika_board.jpg"
    font_path = "font/LightNovelPOPv2.otf"

    options = Options()
    # options.add_argument('--headless')
    # options.add_argument("--no-sandbox") 
    #options.add_argument('--user-agent=' + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36')
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)
    
    url_login = "https://hall-navi.com/login_form_mail"
    browser.implicitly_wait(10)
    
    browser.get(url_login)
    browser.implicitly_wait(10)
    # id
    element = browser.find_element(By.NAME, "id")
    element.click()
    browser.implicitly_wait(10)
    element.send_keys('tsc953u@gmail.com')
    # pw
    element = browser.find_element(By.NAME, "pass")
    element.click()
    browser.implicitly_wait(10)
    element.send_keys('jdhibh')
    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    time.sleep(1)

    browser.refresh()
    todouhuken_number = 1

    today = datetime.date.today()
    tomorrow = datetime.date.today() + datetime.timedelta(days=1)

    # for baitai in ['スロパチ','天草ヤスヲ','ホールサーチマン','Gooパチ','ホール攻略']:
    #     tokyou_syuzai_df_a = tokyou_syuzai_df[tokyou_syuzai_df['baitai'] == baitai]
    #     previous_syuzai = ''p
    #     print('\n'  ,baitai)
    #     for syuzai in tokyou_syuzai_df_a.itertuples():
    #             #syuzai_list.append(syuzai)
    #         if syuzai[1] != previous_syuzai:
    #             print('\n\n■',syuzai[1],'\n   ★',syuzai[2])

    month_tomorrow_str = tomorrow.strftime("%m月").lstrip('0')
    day_tomorrow_str = tomorrow.strftime("%d日").lstrip('0')
    tomorrow_str = month_tomorrow_str + day_tomorrow_str
    tomorrow_url = tomorrow.strftime("%Y-%m-%d")
    output_path = f"eva_board_{tomorrow_url}.jpg"

    url = f"https://hall-navi.com/?area=chubu"
    browser.get(url)
    url = f"https://hall-navi.com/osusume_list?ken={ken_number}&ymd={tomorrow_url}"

    browser.get(url)
    # ログイン後にhtmlを取得する
    html = browser.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "lxml")
    kiji_list = soup.find_all(class_='osbox_out view_osbox')
    time.sleep(1)
    #browser.quit()
    elems = soup.select('.sche_list_contents.osle')
    todouhuken_str = soup.find('h1').text.replace('｜おすすめ総合ランキング', '').replace('｜', '  ')
    h1_text = soup.find('h1').text.replace('｜おすすめ総合ランキング', '').replace('｜', '  ')
    todoufuken_kanji = h1_text.split('  ')[0]

    parlar_name_list:list[str] = []
    syuzai_name:list[str] = []
    tenpo_only_text = []
    chousa_syuzai = []
    context = ''
    ichitenpo_syuzai = []
    #syuzai_df()
    for kiji in kiji_list:
        syuzai_list = kiji.find_all(class_='list_event_name_li')
        if len(syuzai_list) == 0:
            continue
        tenpo_name = kiji.find(class_='oslh2').text
        tenpo_name = tenpo_name.replace('・', '').replace('\n', '').replace(' ', '').replace('　', '')
        print(tenpo_name)
        parlar_name_list.append(tenpo_name)

# except Exception as e:
#     print(e)
#browser.quit()

ライブラリの読み込み完了


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 6.14MB/s]
C:\Users\tsc95\AppData\Local\Temp\ipykernel_19016\3307838891.py:230: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)


キクヤ春日井店
MEGAコンコルド豊川インター店
メガコンコルド1111BLAZE店
メガコンコルド800春日井店+
ミカド一宮インター店
メガコンコルド1177大口41号通り店
MEGAコンコルド1280稲沢店
M&K岡崎店
bingo21
ZENT稲沢店
キャッスル大金店
ZENT木曽川店
ZENT豊田本店
タイキ稲沢駅前店
メガコンコルド1220みなと23号通り店
ラッキー一番江南店
コンコルド800一宮尾西インター店
ミカド稲沢店
がちゃぽん蟹江店
ZENT下市場店
ラッキー1BAN日進竹の山店
タイキ藤沢店
オーギヤ西尾店
yout小牧店
プレイランド名宝
キング観光サウザンド今池2号店
KYORAKU西店
ラッキープラザ名古屋西インター七宝店
マリオン長良橋西店
メガスロットコンコルド345吉浜店
ラッキー一番上飯田店
ZENT扶桑店
大一岩塚店
京楽会館豊明店
プレイランド第一平和


In [64]:
parlar_name_prefecture_num_dict :dict[str:str]= {'愛知県':'23','静岡県':'22','山梨県':\
    '19','長野県':'20','岐阜県':'21','新潟県':'15','富山県':'16','石川県':'17','福井県':'18','三重県':'24'}

In [65]:
parlar_name_list

['キクヤ春日井店',
 'MEGAコンコルド豊川インター店',
 'メガコンコルド1111BLAZE店',
 'メガコンコルド800春日井店+',
 'ミカド一宮インター店',
 'メガコンコルド1177大口41号通り店',
 'MEGAコンコルド1280稲沢店',
 'M&K岡崎店',
 'bingo21',
 'ZENT稲沢店',
 'キャッスル大金店',
 'ZENT木曽川店',
 'ZENT豊田本店',
 'タイキ稲沢駅前店',
 'メガコンコルド1220みなと23号通り店',
 'ラッキー一番江南店',
 'コンコルド800一宮尾西インター店',
 'ミカド稲沢店',
 'がちゃぽん蟹江店',
 'ZENT下市場店',
 'ラッキー1BAN日進竹の山店',
 'タイキ藤沢店',
 'オーギヤ西尾店',
 'yout小牧店',
 'プレイランド名宝',
 'キング観光サウザンド今池2号店',
 'KYORAKU西店',
 'ラッキープラザ名古屋西インター七宝店',
 'マリオン長良橋西店',
 'メガスロットコンコルド345吉浜店',
 'ラッキー一番上飯田店',
 'ZENT扶桑店',
 '大一岩塚店',
 '京楽会館豊明店',
 'プレイランド第一平和']

In [67]:
from sqlalchemy import (Column, String, Text, ForeignKey, \
                create_engine, MetaData, DECIMAL, DATETIME, exc, event, Index)
from sqlalchemy.schema import UniqueConstraint
from sqlalchemy.orm import (sessionmaker, relationship, scoped_session)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import INTEGER as Integer
from datetime import datetime
import os

from sqlalchemy import Column, Integer, create_engine, MetaData, DATETIME, String,Date,Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from pathlib import Path
import os
import datetime 
import pandas as pd

from sqlalchemy import create_engine, text

username = os.getenv('DB_USER_NAME')
password=os.getenv('DB_PASSWORD')
host=os.getenv('DB_HOST')
port='3306'
database=os.getenv('DB_NAME')
db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"

# DB接続するためのEngineインスタンス
engine = create_engine(db_url, echo=False)

# Sessionの作成
session = scoped_session(sessionmaker(autocommit = False,autoflush = False,bind = engine))
# modelで使用する
Base = declarative_base()
Base.query = session.query_property()

query = "select * from schedule where 店舗名 = 'キクヤ春日井店'"
df = pd.read_sql_query(sql=text(query), con=engine.connect())
df.sort_values('イベント日')

,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
5,1249,愛知県,2023-03-15,水,キクヤ春日井店,ナビ子AI予想【差枚プラス】★1 (毎週 水曜日),ホールナビ,A,2023-03-15 11:05:52
9,1923,愛知県,2023-03-15,水,キクヤ春日井店,旧イベ(毎月 15日),旧イベ,・,2023-03-15 11:05:52
13,2933,愛知県,2023-03-15,水,キクヤ春日井店,ナビ子AI予想【差枚プラス】★1 (毎週 水曜日),ホールナビ,A,2023-03-16 11:05:27
14,3088,愛知県,2023-03-15,水,キクヤ春日井店,旧イベ(毎月 15日),旧イベ,・,2023-03-16 11:05:27
0,432,愛知県,2023-03-17,金,キクヤ春日井店,シーサーペントOVER,アツ姫,・,2023-03-15 11:05:52
6,1409,愛知県,2023-03-17,金,キクヤ春日井店,ナビ子AI予想【差枚プラス】★5 (毎週 金曜日),ホールナビ,S,2023-03-15 11:05:52
19,4301,愛知県,2023-03-17,金,キクヤ春日井店,旧イベ(7のつく日),旧イベ,・,2023-03-18 11:06:06
10,2078,愛知県,2023-03-17,金,キクヤ春日井店,旧イベ(7のつく日),旧イベ,・,2023-03-15 11:05:52
18,4161,愛知県,2023-03-17,金,キクヤ春日井店,ナビ子AI予想【差枚プラス】★5 (毎週 金曜日),ホールナビ,S,2023-03-18 11:06:06
17,4065,愛知県,2023-03-17,金,キクヤ春日井店,シーサーペントOVER,アツ姫,・,2023-03-18 11:06:06


In [30]:
date ="2023-03-19"

In [ ]:
pledge_df = pd.read_csv(r'csv\取材公約用マスター.csv')
pledge_df
convert_parlar_name_df = pd.read_csv(r'csv\店舗名変換テーブル.csv')
convert_parlar_name_df = convert_parlar_name_df.rename(columns={'ホールナビ店舗名':'店舗名'})
convert_parlar_name_df

select * from schedule WHERE 店舗名 LIKE '%ラッキープラザ名古屋西インター七宝店%' AND イベント日 = '2023-03-19'


,id,都道府県_x,イベント日,曜日,店舗名,取材名,媒体名_x,取材ランク,取得時間,都道府県_y,店舗kintone,shop_id,DMM店舗名,アナスロ店舗名,緯度,経度,住所,媒体名_y,公約内容
0,1649,愛知県,2023-03-19,日,ラッキープラザ名古屋西インター七宝店,ナビ子AI予想【機種仕掛け】★5 (9のつく日),ホールナビ,S,2023-03-15 11:05:52,愛知県,P12265,12596,ラッキープラザ名古屋西インター七宝店,ラッキープラザ名古屋西インター七宝店,35.156045,136.801224,愛知県あま市七宝町伊福苗代126,ホールナビ,NaN
1,2370,愛知県,2023-03-19,日,ラッキープラザ名古屋西インター七宝店,旧イベ(9のつく日),旧イベ,・,2023-03-15 11:05:52,愛知県,P12265,12596,ラッキープラザ名古屋西インター七宝店,ラッキープラザ名古屋西インター七宝店,35.156045,136.801224,愛知県あま市七宝町伊福苗代126,旧イベ,NaN


In [57]:
from folium import plugins
import branca

In [62]:
import folium

# 地図生成
folium_map = folium.Map(location=[35.1814, 136.9063], zoom_start=15)

# マーカープロット（ポップアップ設定，色変更，アイコン変更）
text = ''
for parlar_name in parlar_name_list[1:10]:
    query = f"select * from schedule WHERE 店舗名 LIKE '%{parlar_name}%' AND イベント日 = '{date}'"
    print(query)
    extract_parlar_name_df = pd.read_sql_query(sql=query, con=engine.connect())
    #display(extract_parlar_name_df )
    #display(convert_parlar_name_df)
    tenpo_syuzai_merge_df = pd.merge(extract_parlar_name_df, convert_parlar_name_df, on='店舗名')
    merge_df = pd.merge(tenpo_syuzai_merge_df, pledge_df, on='取材名')
    merged_df =merge_df[merge_df['都道府県_x'] == '愛知県'].sort_values('取材ランク')
    #display(merged_df)
    
    if len(merged_df) == 0:
        continue
    
    for i,row in merged_df.iterrows():
        #print(row)
        break
    
    popup_df = merged_df[~merged_df['取材名'].str.contains('ナビ子')]
    popup_df = popup_df[['イベント日','曜日','店舗名','公約内容']]
    table_html = branca.element.IFrame(popup_df.to_html(index=False,escape=False), width=500, height=100)
    popup = folium.Popup(
        table_html,
        width=800,
        height=100)
    longitude = row['緯度']
    latitude = row['経度']
    folium.Marker(
        location=[longitude , latitude],
        popup=popup,
        icon=folium.Icon(color='red', icon='home')
    ).add_to(folium_map)

    #break

# 地図表示
folium_map.save(r'html\test.html')
folium_map

select * from schedule WHERE 店舗名 LIKE '%ラッキープラザ名古屋西インター七宝店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%KYORAKU東海店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%メガコンコルド1220みなと23号通り店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%マルシン777%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%ミカド一宮インター店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%プレイランド平和大治店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%グランドオータ888鳴海店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%京楽会館豊明店%' AND イベント日 = '2023-03-19'
select * from schedule WHERE 店舗名 LIKE '%ラッキープラザ大府店%' AND イベント日 = '2023-03-19'


In [ ]:
tenpo_syuzai_merge_df = pd.merge(sql_syuzai_report_all_df, convert_parlar_name_df, on='店舗名')
pmerge_df = pd.merge(tenpo_syuzai_merge_df, pledge_df, on='取材名')
pmerge_df[pmerge_df['都道府県_x'] == '愛知県'].sort_values('イベント日')